# HHL Algorithm

## Imports

In [1]:
import math

import numpy as np

from typing import Optional, Callable, Union, Tuple, List

from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.algorithms import AmplitudeEstimation, EstimationProblem
from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal
from qiskit.circuit.library import PhaseEstimation

from qiskit.opflow import (
    Z,
    I,
    StateFn,
    TensoredOp,
    ExpectationBase,
    CircuitSampler,
    ListOp,
    ExpectationFactory,
    ComposedOp,
)

from linear_solvers.observables.linear_system_observable import LinearSystemObservable
from linear_solvers import LinearSolver, LinearSolverResult, NumPyLinearSolver
from linear_solvers import HHL as TrueHHL
from linear_solvers.matrices.numpy_matrix import NumPyMatrix

/var/folders/j5/zjbd__c92h76zqc6n0yw0msw0000gn/T/ipykernel_11341/1244597687.py:8: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import AmplitudeEstimation, EstimationProblem


A = np.array([
    [1, -1/3],
    [-1/3, 1]
])

b = np.array([1, 0])

nb = int(np.log2(len(b)))

print(f"A = {A}")
print(f"b = {b}")
print(f"Number of qubits = {nb}")


# 1. State Preparation

## Amplitude Estimation

problem = EstimationProblem(
    state_preparation=A,  # A operator
    grover_operator=b,  # Q operator
    objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

compute_bits = QuantumRegister(nb)
ancilla = AncillaRegister(1)
sampler = Sampler()

qc = QuantumCircuit(compute_bits, ancilla)

ae = AmplitudeEstimation(
    num_eval_qubits=nb,  # the number of evaluation qubits specifies circuit width and accuracy
    sampler=sampler,
)

qc.draw()

## resize_matrix

Resizes the matrix to match the appropriate dimensionality. Fills in the new cells with zeros.

- **matrix** np.ndarray: Square ($N \times N$) or vector (1xN)
- **n** int: The desired dimensionality in $2^n$
- **verbose** bool: Verbosity

**returns** np.ndarray: Resized matrix $2^n \times 2^n$

In [2]:
def resize_matrix(matrix: np.ndarray, n: int, verbose: bool = False) -> np.ndarray:
    dim = 2 ** n

    assert matrix.ndim <= 2, "Currently, only 2-Dimensional arrays or lower are supported."

    if verbose:
        print("Dimensions: ", matrix.ndim)

    # Not square or vector
    # if matrix.ndim >= 2:
    #     assert (matrix.shape[0] == matrix.shape[1] and (matrix.shape[0] == 1 or matrix.shape[1] == 1)), "Matrix is not square or vector"
    # else:

    # Check if shape is (1, 2^n) or (2^n, 2^n)
    # n,m = matrix.shape
    new_shape = tuple(dim for _ in range(matrix.ndim))
    R = np.empty(new_shape)
    X=matrix
    
    if matrix.ndim == 1:
        R[:X.shape[0]] = X[:R.shape[0]]
    elif matrix.ndim == 2:
        R[:X.shape[0],:X.shape[1]] = X[:R.shape[0],:R.shape[1]]
    # matrix = np.resize(matrix, [dim for _ in range(matrix.ndim)])

    return R

    # np.resize(matrix,)

In [3]:
test_vector = np.ones(2)
test_m2 = np.ones((2,2))
test_m3 = np.ones((3,3))

assert np.shape(resize_matrix(test_vector, 1)) == test_vector.shape # Test no changes
assert np.shape(resize_matrix(test_vector, 2)) == (2**2,) # Test 2**2

assert resize_matrix(test_m2, 1).shape == test_m2.shape
assert resize_matrix(test_m2, 2).shape == (4,4)

assert resize_matrix(test_m3, 2).shape == (4,4)

assert resize_matrix(test_m2, 4).shape == (2**4, 2**4)

## calc_qubits

Given a vector, calculates the number of qubits needed to encode it into a circuit.

In [4]:
def calc_qubits(vector: np.ndarray) -> int:
    return math.ceil(np.log2(len(vector)))

In [5]:
assert calc_qubits([0, 0]) == 1
assert calc_qubits([0,0,0]) == 2
assert calc_qubits([0,0,0,0,0]) == 3

## AE_Circuit

Amplitude Encoding Circuit.

- **vector** np.ndarray: Vector $b$.
- **nb** int: number of qubits to represent the vector. Optional, and will be calculated otherwise.
- **verbose** bool: Verbosity

**Returns** QuantumCircuit: the quantum circuit representing the amplitude encoding of the vector.

In [6]:
def AE_Circuit(vector: np.ndarray, nb: int = None, verbose: bool = False) -> QuantumCircuit:
    if nb is None:
        nb = calc_qubits(vector)

        if verbose:
            print(f"Number of Qubits: {nb}")

    if len(vector) != 2**nb:
        vector = resize_matrix(vector, nb, verbose=verbose)

    vector_circuit = QuantumCircuit(nb)

    isometry = vector / np.linalg.norm(vector)

    if verbose:
        print("Isometry: ", isometry)

    vector_circuit.isometry(
        isometry, list(range(nb)), None
    )

    vector_circuit.name = "Amplitude Encoding"
        
    return vector_circuit
    

In [7]:
assert AE_Circuit(np.array([1, 0])).num_qubits == 1
assert AE_Circuit(np.array([1,0.1,1])).num_qubits == 2
assert AE_Circuit(np.array([1,0,1,0.2,0.3])).num_qubits == 3

# 2. Quantum Phase Estimation

## create_matrix_circuit


- **matrix** np.ndarray: Input matrix $A$. Should be a square hermitian
- **number_qubits** int: The expected number of qubits for the circuit

**Returns**: A QuantumCircuit representing the matrix A.

In [8]:
def create_matrix_circuit(matrix: np.ndarray, number_qubits: int) -> QuantumCircuit:
    if isinstance(matrix, list):
        matrix = np.array(matrix)

    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Input matrix must be square!")
    if np.log2(matrix.shape[0]) % 1 != 0:
        raise ValueError("Input matrix dimension must be 2^n!")
    if not np.allclose(matrix, matrix.conj().T):
        raise ValueError("Input matrix must be hermitian!")
    if matrix.shape[0] != 2**number_qubits:
        raise ValueError(
            "Input vector dimension does not match input "
            "matrix dimension! Vector dimension: "
            + str(number_qubits)
            + ". Matrix dimension: "
            + str(matrix.shape[0])
        )
    matrix_circuit = NumPyMatrix(matrix, evolution_time=2 * np.pi)

    return matrix_circuit

In [9]:
for i in [1, 2, 3]:
    n = 2**i
    m = np.round(np.random.random((n,n)) * 10,2)
    m = m + m.conj().T
    num_bits = calc_qubits(m)

    circuit = create_matrix_circuit(m, num_bits)

    assert(circuit.num_qubits == num_bits)

    print(circuit)

# Test non-hermitian
m = np.round(np.random.random((2,2)) * 10,2)
try:
    create_matrix_circuit(m, 2)
except ValueError as e:
    assert str(e) == "Input matrix must be hermitian!"

       ┌─────────────┐
state: ┤ circuit-130 ├
       └─────────────┘
         ┌──────────────┐
state_0: ┤0             ├
         │  circuit-134 │
state_1: ┤1             ├
         └──────────────┘
         ┌──────────────┐
state_0: ┤0             ├
         │              │
state_1: ┤1 circuit-138 ├
         │              │
state_2: ┤2             ├
         └──────────────┘


## calculate_nl

Update the number of qubits required to represent the eigenvalues

$$e^{-2 \pi i \lambda} = e^{2 \pi i (1 - \lambda)}$$

- **number_bits** int: The number of qubits to represent the eigenvalues.
- **kappa** float:
- **neg_vals** float: To register negative eigenvalus. Default true

**Returns** float: The value of the scaling factor.

In [10]:
def calculate_nl(number_bits: int, kappa: float, neg_vals: bool = True):
    nl = max(number_bits + 1, int(np.ceil(np.log2(kappa + 1)))) + neg_vals
    return nl

## get_delta

Calculates the scaling factor to represent exactly $\lambda_{min}$ on $n_l$ binary digits.

- **n_l** int: The number of qubits to represent the eigenvalues.
- **lambda_min** float: the smallest eigenvalue.
- **lambda_max** float: the largest eigenvalue.

**Returns** float: The value of the scaling factor.

In [11]:
def get_delta(n_l: int, lambda_min: float, lambda_max: float) -> float:
    formatstr = "#0" + str(n_l + 2) + "b"
    lambda_min_tilde = np.abs(lambda_min * (2**n_l - 1) / lambda_max)
    # floating point precision can cause problems
    if np.abs(lambda_min_tilde - 1) < 1e-7:
        lambda_min_tilde = 1
    binstr = format(int(lambda_min_tilde), formatstr)[2::]
    lamb_min_rep = 0
    
    for i, char in enumerate(binstr):
        lamb_min_rep += int(char) / (2 ** (i + 1))
    return lamb_min_rep

## get_kappa

In [12]:
def get_kappa(matrix_circuit: QuantumCircuit):
    if (
        hasattr(matrix_circuit, "condition_bounds")
        and matrix_circuit.condition_bounds() is not None
    ):
        kappa = matrix_circuit.condition_bounds()[1]
    else:
        kappa = 1

    return kappa

## set_eigenbounds

In [13]:
def set_eigenbounds(matrix_circuit: QuantumCircuit, nl: int, neg_vals: bool = True):
    if (hasattr(matrix_circuit, "eigs_bounds") and matrix_circuit.eigs_bounds() is not None):
        
        lambda_min, lambda_max = matrix_circuit.eigs_bounds()
        # Constant so that the minimum eigenvalue is represented exactly, since it contributes
        # the most to the solution of the system. -1 to take into account the sign qubit
        delta = get_delta(nl - neg_vals, lambda_min, lambda_max)
        # Update evolution time
        matrix_circuit.evolution_time = (
            2 * np.pi * delta / lambda_min / (2**neg_vals)
        )
        # Update the scaling of the solution
        scaling = lambda_min
    else:
        delta = 1 / (2**nl)
        scaling = 1
        print("The solution will be calculated up to a scaling factor.")

    return matrix_circuit, delta, scaling


## calculate_norm

Calculates the value of the euclidean norm of the solution.

- **qc** QuantumCircuit: The quantum circuit preparing the solution x to the system.
- **scaling** float: Scaling factor of the norm

**Returns** float: The value of the euclidean norm of the solution.

In [14]:
def calculate_norm(qc: QuantumCircuit, scaling: float) -> float:
    # Calculate the number of qubits
    nb = qc.qregs[0].size
    nl = qc.qregs[1].size
    na = qc.num_ancillas

    # Create the Operators Zero and One
    zero_op = (I + Z) / 2
    one_op = (I - Z) / 2

    # Norm observable
    observable = one_op ^ TensoredOp((nl + na) * [zero_op]) ^ (I ^ nb)
    norm_2 = (~StateFn(observable) @ StateFn(qc)).eval()

    return np.real(np.sqrt(norm_2) / scaling)

# Production

## HHL

In [15]:
def construct_circuit(A: np.ndarray, b: np.ndarray, neg_vals: bool = True, nf: int = 1, epsilon: float = 1e-2):
    _epsilon_r = epsilon / 3  # conditioned rotation
    _epsilon_s = epsilon / 3  # state preparation
    _epsilon_a = epsilon / 6  # hamiltonian simulation

    nb = calc_qubits(b)
    
    ae = AE_Circuit(b, nb, verbose=True)
    print(ae)

    matrix_circuit = create_matrix_circuit(A, ae.num_qubits) #???

    kappa = get_kappa(matrix_circuit)
    
    nl = calculate_nl(nb, kappa)

    if hasattr(matrix_circuit, "tolerance"):
        matrix_circuit.tolerance = _epsilon_a

    matrix_circuit, delta, scaling = set_eigenbounds(matrix_circuit, nl)

    phase_estimation = PhaseEstimation(nl, matrix_circuit)

    reciprocal_circuit = ExactReciprocal(nl, delta, neg_vals=neg_vals)
    reciprocal_circuit.name = "Conditional Rotation"
    # Update number of ancilla qubits
    na = matrix_circuit.num_ancillas

    # check if the matrix can calculate bounds for the eigenvalues

    # Initialise the quantum registers
    qb = QuantumRegister(nb)  # right hand side and solution
    ql = QuantumRegister(nl)  # eigenvalue evaluation qubits
    if na > 0:
        qa = AncillaRegister(na)  # ancilla qubits
    qf = QuantumRegister(nf)  # flag qubits

    if na > 0:
        qc = QuantumCircuit(qb, ql, qa, qf)
    else:
        qc = QuantumCircuit(qb, ql, qf)
    
    # State preparation
    qc.append(ae, qb[:])
    
    # QPE
    if na > 0:
        qc.append(phase_estimation, ql[:] + qb[:] + qa[: matrix_circuit.num_ancillas])
    else:
        qc.append(phase_estimation, ql[:] + qb[:])

    # Coditional Rotation
    qc.append(reciprocal_circuit, ql[::-1] + [qf[0]])

    # QPE inverse
    if na > 0:
        qc.append(
            phase_estimation.inverse(),
            ql[:] + qb[:] + qa[: matrix_circuit.num_ancillas],
        )
    else:
        qc.append(phase_estimation.inverse(), ql[:] + qb[:])
    return qc, scaling

In [16]:
def HHL(A: np.ndarray, b: np.ndarray, neg_vals: bool = True, nf: int = 1, epsilon: float = 1e-2):
    qc, scaling = construct_circuit(A, b, neg_vals, nf, epsilon)

    norm = calculate_norm(qc, scaling)

    sol = LinearSolverResult()
    
    sol.state = qc
    sol.euclidean_norm = norm

    return sol

In [17]:
A = np.array([
    [1, -1/3],
    [-1/3, 1]
])

b = np.array([1, 0])

sol = HHL(A,b)

Isometry:  [1. 0.]
   ┌──────────┐
q: ┤ Isometry ├
   └──────────┘


/var/folders/j5/zjbd__c92h76zqc6n0yw0msw0000gn/T/ipykernel_11341/2589836013.py:12: DeprecationWarning: The class ``qiskit.opflow.list_ops.tensored_op.TensoredOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  observable = one_op ^ TensoredOp((nl + na) * [zero_op]) ^ (I ^ nb)
/var/folders/j5/zjbd__c92h76zqc6n0yw0msw0000gn/T/ipykernel_11341/2589836013.py:13: DeprecationWarning: The class ``qiskit.opflow.state_fns.operator_state_fn.OperatorStateFn`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  norm_2 = (~StateFn(observable) @ StateFn(qc)).eval()
/var/folders/j5/zjbd__c92h76zqc6n0yw0msw0000gn/T/ipykernel_11341/2589836013.py:13: DeprecationWarning: The class ``qiskit.opflow.state_fns.circuit_state_fn.CircuitStateFn`` is de

In [18]:
compute_x(sol)

NameError: name 'compute_x' is not defined

# Test

In [ ]:
def compute_x(naive_hhl_solution):

    naive_sv = Statevector(naive_hhl_solution.state).data
    
    # Extract vector components; 10000(bin) == 16 & 10001(bin) == 17
    naive_full_vector = np.array([naive_sv[16], naive_sv[17] ])
    
    print('naive raw solution vector:', naive_full_vector)

    def get_solution_vector(solution):
        """Extracts and normalizes simulated state vector
        from LinearSolverResult."""
        solution_vector = Statevector(solution.state).data[16:18].real
        norm = solution.euclidean_norm
        return norm * solution_vector / np.linalg.norm(solution_vector)

    solution = get_solution_vector(naive_hhl_solution)
    print('full naive solution vector:', solution)

    return solution

Now that the functions for the HHL algorithm are defined, I can run it on a number of test cases

In [ ]:
a_tests = [
    np.array([ [1, -1/3], [-1/3, 1] ])
]

b_tests = [
    np.array([1, 0])
]

x_tests = [
    np.array([1.125, 0.375])
]

for n in 2**np.array([1, 2, 3]):
    a = np.round(np.random.random((n,n)) * 10,2)
    b = np.round(np.random.random(n) * 10,2)

    # Make 'a' hermitian
    a = a + a.conj().T

    x = np.linalg.solve(a,b)

    assert np.allclose(np.dot(a, x), b)

    a_tests.append(a)
    b_tests.append(b)
    x_tests.append(x)
print(x_tests)

In [ ]:
for a, b, x in zip(a_tests, b_tests, x_tests):
    naive_hhl_solution = TrueHHL().solve(a, b)
    compute_x(naive_hhl_solution)
    naive_hhl_solution = HHL(a, b, epsilon=1e-4)

    print("X: ", x)
    answer = compute_x(naive_hhl_solution)

    print("Answer: ", answer)
    print(np.allclose(np.dot(a, answer), b))
    print("-----")

In [ ]:

matrix = np.array([ [1, -1/3], [-1/3, 1] ])
vector = np.array([1, 0])
naive_hhl_solution = HHL().solve(A, b)

In [ ]:
naive_hhl_solution.state.draw()

In [ ]:
compute_x(naive_hhl_solution)

In [ ]:
a_tests[-1]

In [ ]:
HHL(np.eye(3), np.ones(3))